<a href="https://colab.research.google.com/github/gseetha04/Transfer-learning/blob/main/Transferlearning_experiments_spanBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##############Fine-tune on SCITE and test on SCITE####################################

In [ ]:
!nvidia-smi

Wed Nov 15 03:55:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 170237, done.
remote: Counting objects: 100% (453/453), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 170237 (delta 238), reused 348 (delta 179), pack-reused 169784
Receiving objects: 100% (170237/170237), 170.00 MiB | 16.10 MiB/s, done.
Resolving deltas: 100% (128681/128681), done.


In [ ]:
cd /content/transformers/examples/pytorch/token-classification/

/content/transformers/examples/pytorch/token-classification


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 32.5 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2yt88jhg
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2yt88jhg
  Resolved https://github.com/huggingface/transformers to commit 2e72bbab2cd169903b1e77b439718c1bdc5d50b2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.2 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8000870 sha256=83415dfc00663643d709e10589fbfb0e1b625b781e6a5530e91b7993797f4dfc
  Stored in directory: /tmp/pip-ephem-wheel-cache-cqotn8gh/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=714e994fb874da5ff55600fe6315ec52ced558d03f42cb238d181603824fe937
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
data_train = pd.read_excel('/content/train_data.xlsx',sheet_name = "Sheet5")
data_train.head(4)

,sentences,Bio_label
0,"[""A"",""rare"",""and"",""incurable"",""congenital"",""di...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""E"",""O"",""O"",""..."
1,"[""The"",""soft"",""brown"",""and"",""beige"",""tones"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""O"",""O"",""E"",""..."
2,"[""The"",""JFK"",""baggage"",""system"",""malfunction"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""..."
3,"[""Cobalt"",""metal"",""fume"",""and"",""dust"",""cause"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""O""]"


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_train[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_train, y_raw_train = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw_train, y_raw_train, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
new_ners = []
for i,j in zip(X_train_raw,y_train_raw):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
new_ners1 = []
for i,j in zip(X_dev_raw,y_dev_raw):
  _new_ner = {'words': i, 'ner': j}
  new_ners1.append(_new_ner)

In [ ]:
def dump_json_lines(jsons, output_dir):
    print(output_dir)
    with open(output_dir, 'w') as outfile:
        for entry in jsons:
            json.dump(entry, outfile)
            outfile.write('\n')

In [ ]:
train = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(train,  '/content/train.json')

/content/train.json


In [ ]:
dev = [new_ners1[i] for i in range(len(new_ners1))]
dump_json_lines(dev,  '/content/dev.json')

/content/dev.json


In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
new_ners = []
for i,j in zip(X_raw_test,y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
!python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
           --train_file /content/train.json  \
           --validation_file /content/dev.json  \
           --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

2023-07-31 16:30:24.382763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/31/2023 16:30:26 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/31/2023 16:30:26 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_c

In [ ]:
import pandas as pd
data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

In [ ]:
data_preds

,preds
0,C C C C C C C O E
1,C C C O E E E E O O C C O E E E E
2,C C C O E E O O O O O O O O O O O O O O O O O ...
3,C C C O E O E O O O O O O O O O
4,C C C O O E O E O O O
...,...
186,O O O O O O O O O O O O O O O O O O O O C C O ...
187,O O O O O O O O O O O O O O O O O O O O E E O ...
188,O O O O O O O O O O O O O O O O O O O O O O O ...
189,O O O O O O O O O O O O O O O O O O O O O O O ...


In [ ]:
data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))

In [ ]:
preds = {'preds':data_preds}

In [ ]:
preds

{'preds':                                                  preds
 0                                    C,C,C,C,C,C,C,O,E
 1                    C,C,C,O,E,E,E,E,O,O,C,C,O,E,E,E,E
 2    C,C,C,O,E,E,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
 3                      C,C,C,O,E,O,E,O,O,O,O,O,O,O,O,O
 4                                C,C,C,O,O,E,O,E,O,O,O
 ..                                                 ...
 186  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,C,C,O,...
 187  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,E,E,O,...
 188  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
 189  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
 190  O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
 
 [191 rows x 1 columns]}

In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test

,words,ner
0,"[Various, hormonal, ,, bacterial, and, inflamm...","[C, C, C, C, C, C, C, O, E]"
1,"[A, stereo, buss, outputs, the, stereo, buss, ...","[C, C, C, O, E, E, E, E, O, C, C, C, O, E, E, ..."
2,"[The, incoming, water, caused, a, stain, on, t...","[C, C, C, O, E, E, O, O, O, O, O, O, O, O, O, ..."
3,"[The, genreal, anesthetic, cause, unconsciousn...","[C, C, C, O, E, O, E, O, O, O, O, O, O, O, O, O]"
4,"[The, alleged, abuse, resulted, in, bruises, a...","[C, C, C, O, O, E, O, E, O, O, O]"
...,...,...
186,"[Thus, ,, evaluating, capital, punishment, as,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
187,"[About, 30, ducks, were, found, dead, in, Klam...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
188,"[He, created, and, advocated, -, flower, power...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
189,"[Method, according, to, claim, 1, ,, character...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
data_test_final = pd.concat([data_test,data_preds],axis=1)

In [ ]:
data_test_final

,words,ner,preds
0,"[Various, hormonal, ,, bacterial, and, inflamm...","[C, C, C, C, C, C, C, O, E]","C,C,C,C,C,C,C,O,E"
1,"[A, stereo, buss, outputs, the, stereo, buss, ...","[C, C, C, O, E, E, E, E, O, C, C, C, O, E, E, ...","C,C,C,O,E,E,E,E,O,O,C,C,O,E,E,E,E"
2,"[The, incoming, water, caused, a, stain, on, t...","[C, C, C, O, E, E, O, O, O, O, O, O, O, O, O, ...","C,C,C,O,E,E,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,"[The, genreal, anesthetic, cause, unconsciousn...","[C, C, C, O, E, O, E, O, O, O, O, O, O, O, O, O]","C,C,C,O,E,O,E,O,O,O,O,O,O,O,O,O"
4,"[The, alleged, abuse, resulted, in, bruises, a...","[C, C, C, O, O, E, O, E, O, O, O]","C,C,C,O,O,E,O,E,O,O,O"
...,...,...,...
186,"[Thus, ,, evaluating, capital, punishment, as,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,C,C,O,..."
187,"[About, 30, ducks, were, found, dead, in, Klam...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,E,E,O,..."
188,"[He, created, and, advocated, -, flower, power...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
189,"[Method, according, to, claim, 1, ,, character...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [ ]:
data_test_final.to_excel('/content/spanbert_gold_preds.xlsx')

In [ ]:
import pandas as pd
data_spanbert_gold_preds = pd.read_excel('/content/spanbert_gold_preds.xlsx')

In [ ]:
flat_list = [i.strip() for i in data_spanbert_gold_preds['ner'] for i in i.split(',')]

In [ ]:
flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

In [ ]:
flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

In [ ]:
import numpy as np
print(np.shape(flat_list))
print(np.shape(flat_list_preds))
print(np.shape(flat_list_words))

(3621,)
(3621,)
(3748,)


In [ ]:
df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

In [ ]:
df_combine = pd.concat([df_finallist_words,df_flatlist,df_finallist],axis=1)

In [ ]:
import pandas as pd
data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

print(data_preds)

preds = {'preds':data_preds}
print(preds)

data_test = pd.read_json('/content/test.json', lines = True)

data_test_final = pd.concat([data_test,data_preds],axis=1)
print(data_test_final)

data_test_final.to_excel('/content/spanbert_gold_preds.xlsx')


                                                 preds
0                                    C C C C C C C O E
1                    C C C O E E E E O O C C O E E E E
2    C C C O E E O O O O O O O O O O O O O O O O O ...
3                      C C C O E O E O O O O O O O O O
4                                C C C O O E O E O O O
..                                                 ...
186  O O O O O O O O O O O O O O O O O O O O C C O ...
187  O O O O O O O O O O O O O O O O O O O O E E O ...
188  O O O O O O O O O O O O O O O O O O O O O O O ...
189  O O O O O O O O O O O O O O O O O O O O O O O ...
190  O O O O O O O O O O O O O O O O O O O O O O O ...

[191 rows x 1 columns]
{'preds':                                                  preds
0                                    C C C C C C C O E
1                    C C C O E E E E O O C C O E E E E
2    C C C O E E O O O O O O O O O O O O O O O O O ...
3                      C C C O E O E O O O O O O O O O
4                              

In [ ]:
import pandas as pd
data_spanbert_gold_preds = pd.read_excel('/content/spanbert_gold_preds.xlsx')

print(data_spanbert_gold_preds)

flat_list = [i.strip() for i in data_spanbert_gold_preds['ner'] for i in i.split(',')]
flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(' ')]
flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]
import numpy as np
print(np.shape(flat_list))
print(np.shape(flat_list_preds))
print(np.shape(flat_list_words))

df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])
df_combine = pd.concat([df_finallist_words,df_flatlist,df_finallist],axis=1)

     Unnamed: 0                                              words  \
0             0  ['Various', 'hormonal', ',', 'bacterial', 'and...   
1             1  ['A', 'stereo', 'buss', 'outputs', 'the', 'ste...   
2             2  ['The', 'incoming', 'water', 'caused', 'a', 's...   
3             3  ['The', 'genreal', 'anesthetic', 'cause', 'unc...   
4             4  ['The', 'alleged', 'abuse', 'resulted', 'in', ...   
..          ...                                                ...   
186         186  ['Thus', ',', 'evaluating', 'capital', 'punish...   
187         187  ['About', '30', 'ducks', 'were', 'found', 'dea...   
188         188  ['He', 'created', 'and', 'advocated', '-', 'fl...   
189         189  ['Method', 'according', 'to', 'claim', '1', ',...   
190         190  ['After', 'the', 'war', ',', 'as', 'the', 'Mid...   

                                                   ner  \
0                            C, C, C, C, C, C, C, O, E   
1    C, C, C, O, E, E, E, E, O, C, C, C, O,

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

              precision    recall  f1-score   support

           C       0.92      0.94      0.93       465
          CE       0.73      0.44      0.55        25
           E       0.96      0.94      0.95       468
           O       0.98      0.99      0.99      2663

    accuracy                           0.97      3621
   macro avg       0.90      0.83      0.86      3621
weighted avg       0.97      0.97      0.97      3621



In [ ]:
#########10 runs######################

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
data_test

,words,ner
0,"[Various, hormonal, ,, bacterial, and, inflamm...","[C, C, C, C, C, C, C, O, E]"
1,"[A, stereo, buss, outputs, the, stereo, buss, ...","[C, C, C, O, E, E, E, E, O, C, C, C, O, E, E, ..."
2,"[The, incoming, water, caused, a, stain, on, t...","[C, C, C, O, E, E, O, O, O, O, O, O, O, O, O, ..."
3,"[The, genreal, anesthetic, cause, unconsciousn...","[C, C, C, O, E, O, E, O, O, O, O, O, O, O, O, O]"
4,"[The, alleged, abuse, resulted, in, bruises, a...","[C, C, C, O, O, E, O, E, O, O, O]"
...,...,...
186,"[Thus, ,, evaluating, capital, punishment, as,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
187,"[About, 30, ducks, were, found, dead, in, Klam...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
188,"[He, created, and, advocated, -, flower, power...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
189,"[Method, according, to, claim, 1, ,, character...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
data_test

,words,ner,liststring
0,"[Various, hormonal, ,, bacterial, and, inflamm...","[C, C, C, C, C, C, C, O, E]","C,C,C,C,C,C,C,O,E"
1,"[A, stereo, buss, outputs, the, stereo, buss, ...","[C, C, C, O, E, E, E, E, O, C, C, C, O, E, E, ...","C,C,C,O,E,E,E,E,O,C,C,C,O,E,E,E,E"
2,"[The, incoming, water, caused, a, stain, on, t...","[C, C, C, O, E, E, O, O, O, O, O, O, O, O, O, ...","C,C,C,O,E,E,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,"[The, genreal, anesthetic, cause, unconsciousn...","[C, C, C, O, E, O, E, O, O, O, O, O, O, O, O, O]","C,C,C,O,E,O,E,O,O,O,O,O,O,O,O,O"
4,"[The, alleged, abuse, resulted, in, bruises, a...","[C, C, C, O, O, E, O, E, O, O, O]","C,C,C,O,O,E,O,E,O,O,O"
...,...,...,...
186,"[Thus, ,, evaluating, capital, punishment, as,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,C,C,O,..."
187,"[About, 30, ducks, were, found, dead, in, Klam...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,E,E,O,..."
188,"[He, created, and, advocated, -, flower, power...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
189,"[Method, according, to, claim, 1, ,, character...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [ ]:
import numpy as np

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-12 19:15:46.209793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 19:15:46.209849: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 19:15:46.209890: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 19:15:47.743515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/12/2023 19:15:51 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/12/2023 19:15:51 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
######################SCITE as train and Organizational data as test###########################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final.xlsx")

Mounted at /content/drive


In [ ]:
import pandas as pd
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
RANDOM_SEED=42

In [ ]:
import pandas as pd
data = pd.read_excel(data_path) #'/content/BERT_data_final.xlsx',sheet_name='Sheet2'
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO Code_changed": "y"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw_test[i][j] = a.replace('CT', 'CE')

In [ ]:
new_ners2 = []
for i,j in zip(X_raw_test, y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners2.append(_new_ner)

In [ ]:
test = [new_ners2[i] for i in range(len(new_ners2))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
np.shape(test)

(2234,)

In [ ]:
!python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
           --train_file /content/train.json  \
           --validation_file /content/dev.json  \
           --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

2023-07-31 17:49:22.598251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
07/31/2023 17:49:25 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/31/2023 17:49:25 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_c

In [ ]:
import pandas as pd
data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

print(data_preds)

preds = {'preds':data_preds}
print(preds)

data_test = pd.read_json('/content/test.json', lines = True)

data_test_final = pd.concat([data_test,data_preds],axis=1)
print(data_test_final)

data_test_final.to_excel('/content/spanbert_gold_preds.xlsx')

                                                  preds
0               O O O O O O E O E O O O O O O O O O O O
1     O O O O O O C O C O O O O O O O O O O O O O O ...
2     O O O O O O O O E E O O E E O O O O O O O O O ...
3     O O O O O O O O O O O O O O O O O O O O O O O ...
4     O O O C C O O O O O O O O O C O O O O O O O O ...
...                                                 ...
2229   E E O O O O O O O O CE CE CE O O C C O C C O O O
2230                          E E E E O O C C O O O O O
2231                    E E O E E O O O O O C C C C C C
2232                    E E E E O O C C O O O O C O O O
2233  E E E E O O C C O E E O E E E E O O O O O C C C C

[2234 rows x 1 columns]
{'preds':                                                   preds
0               O O O O O O E O E O O O O O O O O O O O
1     O O O O O O C O C O O O O O O O O O O O O O O ...
2     O O O O O O O O E E O O E E O O O O O O O O O ...
3     O O O O O O O O O O O O O O O O O O O O O O O ...
4     O O O C

In [ ]:
import pandas as pd
data_spanbert_gold_preds = pd.read_excel('/content/spanbert_gold_preds.xlsx')

print(data_spanbert_gold_preds)

flat_list = [i.strip() for i in data_spanbert_gold_preds['ner'] for i in i.split(',')]
flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(' ')]
flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]
import numpy as np
print(np.shape(flat_list))
print(np.shape(flat_list_preds))
print(np.shape(flat_list_words))

df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])
df_combine = pd.concat([df_finallist_words,df_flatlist,df_finallist],axis=1)

      Unnamed: 0                                              words  \
0              0  ['When ', 'a ', 'policyholder ', 'or ', 'insur...   
1              1  ['During ', '2020 ', 'in ', 'response ', 'to '...   
2              2  ['Prolonged ', 'periods ', 'of ', 'low ', 'int...   
3              3  ['Conversely ', 'a ', 'rise ', 'in ', 'interes...   
4              4  ['Further ', 'because ', 'of ', 'the ', 'conce...   
...          ...                                                ...   
2229        2229  ['The ', 'increase ', 'in ', 'adjusted ', 'PPN...   
2230        2230  ['Modest ', 'noninterest ', 'income ', 'growth...   
2231        2231  ['Wealth ', 'management ', 'and ', 'trust ', '...   
2232        2232  ['The ', '2019 ', 'provision ', 'increased ', ...   
2233        2233  ['The ', '2019 ', 'provision ', 'increased ', ...   

                                                    ner  \
0     O, O, C, C, C, C, C, C, C, O, O, CE, E, E, E, ...   
1     O, O, O, O, O, C, C, C,

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

              precision    recall  f1-score   support

           C       0.74      0.16      0.27     28263
          CE       0.00      0.00      0.00      4436
           E       0.68      0.13      0.22     28549
           O       0.17      0.89      0.29     12035

    accuracy                           0.26     73283
   macro avg       0.40      0.30      0.20     73283
weighted avg       0.58      0.26      0.24     73283



In [ ]:
#######10 runs

In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test

,words,ner
0,"[When , a , policyholder , or , insured , gets...","[O, O, C, C, C, C, C, C, C, O, O, CE, E, E, E,..."
1,"[During , 2020 , in , response , to , the , on...","[O, O, O, O, O, C, C, C, C, C, C, C, C, C, C, ..."
2,"[Prolonged , periods , of , low , interest , r...","[C, C, C, C, C, C, O, CE, E, E, O, O, O, CE, O..."
3,"[Conversely , a , rise , in , interest , rates...","[O, O, C, C, C, C, O, CE, O, O, O, O, O, O, E,..."
4,"[Further , because , of , the , concentration ...","[O, CE, O, O, C, C, C, C, C, C, C, C, C, C, C,..."
...,...,...
2229,"[The , increase , in , adjusted , PPNR , was ,...","[O, E, E, E, E, E, E, E, E, E, E, E, E, CE, CE..."
2230,"[Modest , noninterest , income , growth , due ...","[E, E, E, E, CE, CE, C, C, C, C, C, C, C]"
2231,"[Wealth , management , and , trust , fees , in...","[E, E, E, E, E, E, CE, CE, CE, O, C, C, C, C, ..."
2232,"[The , 2019 , provision , increased , due , to...","[O, O, E, E, CE, CE, C, C, C, C, C, O, C, C, C..."


In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-12 20:09:13.694125: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 20:09:13.694199: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 20:09:13.694243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 20:09:15.047657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/12/2023 20:09:17 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/12/2023 20:09:17 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
######################SCITE as train and fincausal data as test###########################

In [ ]:
#!unzip '/content/data.zip' -d '/content/'

In [ ]:
#/content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.csv
#data_test = pd.read_json('/content/task2_2022_addition-dev.json',lines=True)   #/content/data/fin_causal_2022/new_data

In [ ]:
# feedback_dict = {'B-Effect': 'E', 'I-Effect': 'E', 'B-Cause': 'C', 'I-Cause': 'C'}
# data_test['ner_changed'] = data_test['ner'].apply(lambda x : [feedback_dict.get(i,i)  for i in list(x)])
#data_test['ner_changed'].replace(, regex=True)

In [ ]:
#data_test

,words,ner,id,postfix,ner_changed
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",NaN,NaN,"[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4.0,"[ , , , , , , , , , , , , , , , ...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7.0,"[ , , , , , , , , , , , , , , , , ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8.0,"[ , , , , , , , , , , , , , , , ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
4,"[In, addition, to, the, impacts, from, the, Me...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",9.0,"[ , , , , , , , , , , , , , , , ,...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
...,...,...,...,...,...
260,"[In, addition, ,, corporate, and, other, gener...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, O, ...",526.0,"[ , , , , , , , , , , , , , , , , ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, O, ..."
261,"[The, provision, for, credit, losses, increase...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",528.0,"[ , , , , , , , , , , , , , , , , , ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
262,"[Depreciation, and, amortization, increased, p...","[E, E, E, E, O, O, O, C, C, C, C, C, C, C, C, ...",531.0,"[ , , , , , , , , , , , , , , , ...","[E, E, E, E, O, O, O, C, C, C, C, C, C, C, C, ..."
263,"[We, recorded, a, loss, of, $, 13.1, million, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",534.0,"[ , , , , , , , , , , , , , , , ,...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."


In [ ]:
#data_test.to_excel('/content/fincausal_testdata.xlsx')

In [ ]:
import pandas as pd
data_fincausal_test = pd.read_json('/content/Fincausal-test.json', lines=True)

In [ ]:
data_fincausal_test

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."
4,"[In, addition, to, the, impacts, from, the, Me...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",9,"[ , , , , , , , , , , , , , , , ,..."
...,...,...,...,...
260,"[In, addition, ,, corporate, and, other, gener...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, O, ...",526,"[ , , , , , , , , , , , , , , , , ..."
261,"[The, provision, for, credit, losses, increase...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",528,"[ , , , , , , , , , , , , , , , , , ..."
262,"[Depreciation, and, amortization, increased, p...","[E, E, E, E, O, O, O, C, C, C, C, C, C, C, C, ...",531,"[ , , , , , , , , , , , , , , , ..."
263,"[We, recorded, a, loss, of, $, 13.1, million, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",534,"[ , , , , , , , , , , , , , , , ,..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
new_ners2 = []
for i,j in zip(X_raw_test, y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners2.append(_new_ner)

In [ ]:
test = [new_ners2[i] for i in range(len(new_ners2))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test

,words,ner,liststring
0,"[When , a , policyholder , or , insured , gets...","[O, O, C, C, C, C, C, C, C, O, O, CE, E, E, E,...","O,O,C,C,C,C,C,C,C,O,O,CE,E,E,E,E,E,E,E,E"
1,"[During , 2020 , in , response , to , the , on...","[O, O, O, O, O, C, C, C, C, C, C, C, C, C, C, ...","O,O,O,O,O,C,C,C,C,C,C,C,C,C,C,O,O,CE,E,E,E,E,O..."
2,"[Prolonged , periods , of , low , interest , r...","[C, C, C, C, C, C, O, CE, E, E, O, O, O, CE, O...","C,C,C,C,C,C,O,CE,E,E,O,O,O,CE,O,O,O,CE,O,CE,O,..."
3,"[Conversely , a , rise , in , interest , rates...","[O, O, C, C, C, C, O, CE, O, O, O, O, O, O, E,...","O,O,C,C,C,C,O,CE,O,O,O,O,O,O,E,E,E,E,E,E,E,CE,..."
4,"[Further , because , of , the , concentration ...","[O, CE, O, O, C, C, C, C, C, C, C, C, C, C, C,...","O,CE,O,O,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,O,O..."
...,...,...,...
2229,"[The , increase , in , adjusted , PPNR , was ,...","[O, E, E, E, E, E, E, E, E, E, E, E, E, CE, CE...","O,E,E,E,E,E,E,E,E,E,E,E,E,CE,CE,C,C,O,C,C,C,C,C"
2230,"[Modest , noninterest , income , growth , due ...","[E, E, E, E, CE, CE, C, C, C, C, C, C, C]","E,E,E,E,CE,CE,C,C,C,C,C,C,C"
2231,"[Wealth , management , and , trust , fees , in...","[E, E, E, E, E, E, CE, CE, CE, O, C, C, C, C, ...","E,E,E,E,E,E,CE,CE,CE,O,C,C,C,C,C,C"
2232,"[The , 2019 , provision , increased , due , to...","[O, O, E, E, CE, CE, C, C, C, C, C, O, C, C, C...","O,O,E,E,CE,CE,C,C,C,C,C,O,C,C,C,C"


In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-12 21:00:38.980077: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 21:00:38.980135: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 21:00:38.980195: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 21:00:41.051407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/12/2023 21:00:45 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/12/2023 21:00:45 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
# !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
#            --train_file /content/train.json  \
#            --validation_file /content/dev.json  \
#            --test_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-full-test.json  \
#            --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

In [ ]:
################ FinCausal as train and FinCausal as test#########################################

In [ ]:
# !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
#            --train_file /content/content/data/fin_causal_2022/fincausal-extra-all-train.json  \
#            --validation_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
#            --test_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
#            --output_dir /content/content/data/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16  #--classifier_dropout 0.1

In [ ]:
#in FinCausal.ipynb at gseetha04

In [ ]:
# !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased   \
#            --train_file /content/content/data/fin_causal_2022/fincausal-extra-all-train.json  \
#            --validation_file /content/content/data/fin_causal_2022/new_data/task2_2022_addition-dev.json  \
#            --test_file /content/test.json  \
#            --output_dir /content/test-ner --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16  #--classifier_dropout 0.1

In [ ]:
import json

In [ ]:
import pandas as pd
data_train = pd.read_json('/content/fincausal-extra-all-train.json', lines=True)
data_train.head(4)

,words,ner,id,postfix
0,"[It, found, that, total, U.S., health, care, s...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0001.00026,"[ , , , , , , , , , , , , , , , ..."
1,"[Transat, loss, more, than, doubles, as, it, w...","[E, E, E, E, E, O, C, C, C, C, C, C, C]",0003.00001,"[ , , , , , , , , , , , ]"
2,"[MONTREAL, -, Transat, AT, Inc., 's, third-qua...","[O, O, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00002,"[ , , , , , , , , , , , , , , ,..."
3,"[The, Montreal-based, company, lost, $, 11.0, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00003,"[ , , , , , , , , , , , , , , , ,..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_train = data_train[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_train.A.values], [json.dumps(labels_test) for labels_test in dataset_train.B.values]

X_raw, y_raw = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
new_ners = []
for i,j in zip(X_raw,y_raw):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
train = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(train,  '/content/train.json')

/content/train.json


In [ ]:
new_ners = []
for i,j in zip(X_raw_test,y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/dev.json')

/content/dev.json


In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
test

[{'words': ['During',
   '2020',
   ',',
   'the',
   'Company',
   'also',
   'issued',
   '$',
   '6.5',
   'billion',
   'of',
   'senior',
   'and',
   'subordinated',
   'long-term',
   'debt',
   '.',
   'Total',
   'shareholders',
   "'",
   'equity',
   'was',
   '$',
   '70.9',
   'billion',
   'at',
   'December',
   '31',
   ',',
   '2020',
   ',',
   'up',
   '$',
   '4.4',
   'billion',
   'compared',
   'to',
   'the',
   'prior',
   'year',
   '.',
   'The',
   'increase',
   'is',
   'due',
   'to',
   'net',
   'income',
   'in',
   'excess',
   'of',
   'dividends',
   'paid',
   'of',
   '$',
   '1.8',
   'billion',
   'and',
   'OCI',
   'of',
   '$',
   '1.6',
   'billion',
   '.'],
  'ner': ['E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',
   'E',


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test

,words,ner
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
4,"[In, addition, to, the, impacts, from, the, Me...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
...,...,...
260,"[In, addition, ,, corporate, and, other, gener...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, O, ..."
261,"[The, provision, for, credit, losses, increase...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."
262,"[Depreciation, and, amortization, increased, p...","[E, E, E, E, O, O, O, C, C, C, C, C, C, C, C, ..."
263,"[We, recorded, a, loss, of, $, 13.1, million, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ..."


In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
data_test

,words,ner,liststring
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...","E,E,E,E,E,E,E,E,O,O,O,O,C,C,C,C,C,C,C,C,C,C,C,..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,C,C,C,..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,O,O,O,O,C,C,C"
4,"[In, addition, to, the, impacts, from, the, Me...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,O,O,C,C,C,C,C"
...,...,...,...
260,"[In, addition, ,, corporate, and, other, gener...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, O, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,O,O,O,O,C,C,C,C,C,..."
261,"[The, provision, for, credit, losses, increase...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,..."
262,"[Depreciation, and, amortization, increased, p...","[E, E, E, E, O, O, O, C, C, C, C, C, C, C, C, ...","E,E,E,E,O,O,O,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,C,..."
263,"[We, recorded, a, loss, of, $, 13.1, million, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...","E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,O,..."


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-14 18:01:27.043520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 18:01:27.043573: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 18:01:27.043622: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 18:01:28.135033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/14/2023 18:01:31 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2023 18:01:31 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
########Fincausal as train and SCITE as test#################################

In [ ]:
#SCITE, SCITE - 0.94
#SCITE, Fincausal - 0.25
#SCITE, Organizational - 0.25
#Fincausal, SCITE -
#Fincausal, Organizational -
#Fincausal, Fincausal - 0.96
#Organizational, SCITE - 0.37
#Organizational, Fincausal - 0.70
#Organizational, Organizational - 0.87

In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'O')

In [ ]:
new_ners = []
for i,j in zip(X_raw_test,y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/dev.json')

/content/dev.json


In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test

,words,ner
0,"[Various, hormonal, ,, bacterial, and, inflamm...","[C, C, C, C, C, C, C, O, E]"
1,"[A, stereo, buss, outputs, the, stereo, buss, ...","[C, C, C, O, E, E, E, E, O, C, C, C, O, E, E, ..."
2,"[The, incoming, water, caused, a, stain, on, t...","[C, C, C, O, E, E, O, O, O, O, O, O, O, O, O, ..."
3,"[The, genreal, anesthetic, cause, unconsciousn...","[C, C, C, O, E, O, E, O, O, O, O, O, O, O, O, O]"
4,"[The, alleged, abuse, resulted, in, bruises, a...","[C, C, C, O, O, E, O, E, O, O, O]"
...,...,...
186,"[Thus, ,, evaluating, capital, punishment, as,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
187,"[About, 30, ducks, were, found, dead, in, Klam...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
188,"[He, created, and, advocated, -, flower, power...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
189,"[Method, according, to, claim, 1, ,, character...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-14 19:43:32.867107: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 19:43:32.867162: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 19:43:32.867207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 19:43:34.072319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/14/2023 19:43:37 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2023 19:43:37 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
#############FinCausal as train and Organizational as test##############################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
# # Our data is split into sentences


# # FIXME: we also need test data!
# X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw_test[i][j] = a.replace('CT', 'O')

In [ ]:
new_ners2 = []
for i,j in zip(X_raw_test, y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners2.append(_new_ner)

In [ ]:
test = [new_ners2[i] for i in range(len(new_ners2))]
dump_json_lines(test,  '/content/dev.json')

/content/dev.json


In [ ]:
test = [new_ners2[i] for i in range(len(new_ners2))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-14 23:40:12.413688: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-14 23:40:12.413746: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-14 23:40:12.413776: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-14 23:40:13.866586: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/14/2023 23:40:16 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/14/2023 23:40:16 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
#######Organizational data as train and test##############################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw[i][j] = a.replace('CT', 'O')

In [ ]:
# from sklearn.model_selection import train_test_split
# RANDOM_SEED = 42
# # Our data is split into sentences


# # FIXME: we also need test data!
# X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
# import numpy as np
# X_train_final = np.array(X_train_raw)
# y_train_final = np.array(y_train_raw)

In [ ]:
# import os
# from sklearn.model_selection import KFold
# k=2
# seed = 1
# kf = KFold(n_splits=k,random_state=seed, shuffle=True)
# metrics=[]
# i=0
# for train_index, val_index in kf.split(X_train_final,y_train_final):
#   train_texts, test_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
#   train_labels, test_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
# import numpy as np
# X_train_final = np.array(X_raw)
# y_train_final = np.array(y_raw)

In [ ]:
new_ners = []
for i,j in zip(X_raw,y_raw):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
train = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(train,  '/content/train.json')

/content/train.json


In [ ]:
new_ners = []
for i,j in zip(X_dev_raw,y_dev_raw):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
dev = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(dev,  '/content/dev.json')

/content/dev.json


In [ ]:
dev = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(dev,  '/content/test.json')

/content/test.json


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-15 15:06:56.959877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 15:06:56.959929: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 15:06:56.959973: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 15:06:58.013272: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/15/2023 15:07:00 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/15/2023 15:07:00 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
##############Organizational as train and SCITE as test##############################

In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
import json
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'CT')

In [ ]:
new_ners = []
for i,j in zip(X_raw_test,y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners.append(_new_ner)

In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/dev.json')

/content/dev.json


In [ ]:
test = [new_ners[i] for i in range(len(new_ners))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-15 16:52:04.673002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 16:52:04.673052: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 16:52:04.673082: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 16:52:05.829841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/15/2023 16:52:08 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/15/2023 16:52:08 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,

In [ ]:
#################Organizational as train and FinCausal as test #######################

In [ ]:
import pandas as pd
data_fincausal_test = pd.read_json('/content/Fincausal-test.json', lines=True)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_fincausal_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]


In [ ]:
new_ners2 = []
for i,j in zip(X_raw_test, y_raw_test):
  _new_ner = {'words': i, 'ner': j}
  new_ners2.append(_new_ner)

In [ ]:
test = [new_ners2[i] for i in range(len(new_ners2))]
dump_json_lines(test,  '/content/dev.json')

/content/dev.json


In [ ]:
test = [new_ners2[i] for i in range(len(new_ners2))]
dump_json_lines(test,  '/content/test.json')

/content/test.json


In [ ]:
data_test = pd.read_json('/content/test.json',lines=True)

In [ ]:
data_test['liststring'] = [','.join(map(str, l)) for l in data_test['ner']]

In [ ]:
for i in range(10):
  !python run_ner.py --model_name_or_path SpanBERT/spanbert-large-cased --train_file /content/train.json --validation_file /content/dev.json --test_file /content/test.json  \
           --output_dir /content/test-ner-2022 --do_train --do_eval --do_predict --learning_rate 5e-5 --num_train_epochs 30 --weight_decay 0 --per_device_train_batch_size 16

  data_preds = pd.read_csv('/content/test-ner-2022/predictions.txt', sep='\t',names=['preds'])

  data_preds = data_preds.apply(lambda x: x.str.replace(' ',','))
  preds = {'preds':data_preds}
  print(preds)

  #data_test = pd.read_json('/content/test.json',lines=True)

  data_test_final = pd.concat([data_test['liststring'],data_preds],axis=1)

  # data_test_final = data_test_final.apply(lambda x: x.replace('[',''))
  # data_test_final = data_test_final.apply(lambda x: x.str.replace(']',''))
  # data_test_final['ner'] = data_test_final['ner'].replace('[','')
  # data_test_final['ner'] = data_test_final['ner'].replace(']','')
  print(data_test_final)

  data_spanbert_gold_preds = data_test_final

  flat_list = [i.strip() for i in data_spanbert_gold_preds['liststring'] for i in i.split(',')]

  flat_list_preds = [i.strip() for i in data_spanbert_gold_preds['preds'] for i in i.split(',')]

 # flat_list_words = [i.strip() for i in data_spanbert_gold_preds['words'] for i in i.split(',')]

  print(np.shape(flat_list))
  print(np.shape(flat_list_preds))
 # print(np.shape(flat_list_words))

  df_flatlist = pd.DataFrame(flat_list,columns=['gold'])
  df_finallist = pd.DataFrame(flat_list_preds,columns=['preds'])
 # df_finallist_words = pd.DataFrame(flat_list_words,columns=['words'])

  df_combine = pd.concat([df_flatlist,df_finallist],axis=1)

  print(classification_report(flat_list, flat_list_preds, zero_division = 1)) #, target_names=target_names

2023-11-15 19:50:49.753767: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 19:50:49.753813: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 19:50:49.753860: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 19:50:50.851135: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/15/2023 19:50:53 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/15/2023 19:50:53 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,